In [1]:
from dulwich.repo import Repo
from dulwich import porcelain
import difflib
import json
import mimetypes

In [2]:
def can_get_file(file):
    try:
        file.data.decode()
        return True
    except UnicodeDecodeError:
        return False

In [3]:
def get_diff(repo, entry):
    res = {}
    for c in entry.changes():
        res[str(c.new.path)] = {}

        d = difflib.Differ()
        #mime = str(mimetypes.guess_type(r.get_object(c.old.sha)))
        #print(mime)
        #if can_get_file(repo.get_object(c.old.sha)):# and can_get_file(r.get_object(c.new.sha)):
        diffs = d.compare(repo.get_object(c.old.sha).data.decode().splitlines(),\
                      repo.get_object(c.new.sha).data.decode().splitlines())

        added = 0
        deleted = 0
        for j in diffs:
            if j[0] =='+':
                added += 1
            elif j[0] =='-':
                deleted += 1

        res[str(c.new.path)]['added'] = added
        res[str(c.new.path)]['deleted'] = deleted
    return res

In [4]:
def form(repo):
    res = {}
    i=0
    
    for c in repo.get_walker():
        if i==10:
            break
        if len(c.commit.parents) <= 1:
            res[str(c.commit.author)] = {} 
            #res[c.commit.author][c.commit.sha().hexdigest()] = {}
            res[str(c.commit.author)][str(c.commit.sha().hexdigest())] = get_diff(repo, c)
        i += 1
            
    return res

In [5]:
def repo_to_json(name):
    r = Repo(name)
    res = json.JSONEncoder().encode(form(r))

    with open('res.json','w') as f:
        f.write(res)

In [6]:
repo_to_json('scikit-learn')